In [ ]:
import pandas as pd
import numpy as np
import math

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/aaditkapoor/tic-tac-toe-ml-project/master/tic-tac-toe.data.txt")

In [ ]:
df.head()

,x,x.1,x.2,x.3,o,o.1,x.4,o.2,o.3,positive
0,x,x,x,x,o,o,o,x,o,positive
1,x,x,x,x,o,o,o,o,x,positive
2,x,x,x,x,o,o,o,b,b,positive
3,x,x,x,x,o,o,b,o,b,positive
4,x,x,x,x,o,o,b,b,o,positive


In [ ]:
df = df.sample(frac=1)
df_test = df[len(df)-200:]
df = df[:len(df)-200]

In [ ]:
df.rename(columns={"x":"c1","x.1":"c2","x.2":"c3","x.3":"c4","o":"c5","o.1":"c6","x.4":"c7","o.2":"c8","o.3":"c9"},inplace=True)

In [ ]:
df.head()

,c1,c2,c3,c4,c5,c6,c7,c8,c9,positive
0,x,x,x,x,o,o,o,x,o,positive
1,x,x,x,x,o,o,o,o,x,positive
2,x,x,x,x,o,o,o,b,b,positive
3,x,x,x,x,o,o,b,o,b,positive
4,x,x,x,x,o,o,b,b,o,positive


In [ ]:
cols = list(set(df.columns)-{"positive"})
for i in cols:
  df[i] = df[i].apply(lambda x: "x" if x=="b" else x)

In [ ]:
df["positive"].value_counts()

positive    625
negative    332
Name: positive, dtype: int64

<h4>Probabilistic Measure</h4> <br>
P(positive|c1,c2...c9) = ℿ(c<sub>i</sub> | positive) * P(positive)

In [ ]:
#Probability of positive
Pos = df["positive"].value_counts()[0]
P_pos = Pos/len(df)

#Probability of c1 given positive
Prob_Ci = {}
k=0
for i in list(set(df.columns)-{"positive"}):
  pos_x = len(df[(df[i]=='x') & (df["positive"]=="positive")])/Pos
  pos_o = len(df[(df[i]=='o') & (df["positive"]=="positive")])/Pos
  k+=1
  Prob_Ci[i] = {"x":pos_x,"o":pos_o}

print(Prob_Ci)

{'c4': {'x': 0.6462167689161554, 'o': 0.3537832310838446}, 'c5': {'x': 0.7566462167689162, 'o': 0.24335378323108384}, 'c2': {'x': 0.6257668711656442, 'o': 0.37423312883435583}, 'c6': {'x': 0.6339468302658486, 'o': 0.3660531697341513}, 'c7': {'x': 0.7116564417177914, 'o': 0.2883435582822086}, 'c9': {'x': 0.6993865030674846, 'o': 0.3006134969325153}, 'c8': {'x': 0.6380368098159509, 'o': 0.3619631901840491}, 'c1': {'x': 0.6871165644171779, 'o': 0.3128834355828221}, 'c3': {'x': 0.689161554192229, 'o': 0.310838445807771}}


<h4>Probabilistic Measure</h4> <br>
P(negative|c1,c2...c9) = ℿ(c<sub>i</sub> | negative) * P(negative)

In [ ]:
#Probability of positive
Neg = df["positive"].value_counts()[1]
P_neg = Neg/len(df)

#Probability of c1 given positive
Prob_Ci_neg = {}
for i in list(set(df.columns)-{"positive"}):
  neg_x = len(df[(df[i]=='x') & (df["positive"]=="negative")])/Neg
  neg_o = len(df[(df[i]=='o') & (df["positive"]=="negative")])/Neg

  Prob_Ci_neg[i] = {"x":neg_x,"o":neg_o}

print(Prob_Ci_neg)

{'c4': {'x': 0.7014925373134329, 'o': 0.29850746268656714}, 'c5': {'x': 0.4291044776119403, 'o': 0.5708955223880597}, 'c2': {'x': 0.6940298507462687, 'o': 0.30597014925373134}, 'c6': {'x': 0.6977611940298507, 'o': 0.30223880597014924}, 'c7': {'x': 0.5410447761194029, 'o': 0.458955223880597}, 'c9': {'x': 0.5597014925373134, 'o': 0.44029850746268656}, 'c8': {'x': 0.6865671641791045, 'o': 0.31343283582089554}, 'c1': {'x': 0.582089552238806, 'o': 0.417910447761194}, 'c3': {'x': 0.5485074626865671, 'o': 0.45149253731343286}}


In [ ]:
cols = list(set(df.columns)-{"positive"})
X = df_test[cols]

In [ ]:
yhat = []
for i in X.to_numpy():
  temp_pos = P_pos
  temp_neg = P_neg
  for j in range(len(cols)):
    #print(i[j])
    temp_pos*=Prob_Ci[cols[j]][i[j]]
    temp_neg*=Prob_Ci_neg[cols[j]][i[j]]

  Prob_pos = (temp_pos*100)/(temp_pos+temp_neg)
  Prob_neg = (temp_neg*100)/(temp_pos+temp_neg)

  if Prob_pos>Prob_neg:
    yhat.append("positive")
  else:
    yhat.append("negative")

In [ ]:
correct = 0
for i in range(len(yhat)):
  if yhat[i]==df_test["positive"][i]:
    correct+=1

print("Accuracy:",round(correct/len(df_test),2))

Accuracy: 0.69


In [ ]:
df_test
df_test.index = np.arange(0, len(df_test))

In [ ]:
df_test

,c1,c2,c3,c4,c5,c6,c7,c8,c9,positive
0,o,x,x,x,x,x,o,o,x,positive
1,x,x,o,x,x,x,x,o,x,positive
2,x,x,x,x,x,x,o,o,o,negative
3,o,x,x,x,x,o,x,o,x,positive
4,x,x,o,x,x,o,x,x,o,negative
...,...,...,...,...,...,...,...,...,...,...
95,x,o,x,x,o,x,x,o,x,negative
96,o,x,x,x,o,x,x,o,o,negative
97,o,x,o,x,o,x,o,x,x,negative
98,o,x,x,x,o,x,x,x,o,negative
